<a href="https://colab.research.google.com/github/GNuSeekK/ICTCOG/blob/main/4_07_%5B%EB%94%A5%EB%9F%AC%EB%8B%9D%5D_%EB%AF%B8%EB%8B%88%EB%B0%B0%EC%B9%98_%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 배치란?
* 데이터의 묶음
* 묶음 대로 결과물이 병렬로 계산 된다.
    * 100개의 데이터를 한꺼번에 묶어서(배치를 만들어서) 입력을 했으면, 거기에 대한 결과물도 100개가 한꺼번에 나온다.
* 배치를 적용한 Loss의 수식은?
* 배치가 `N`건이면, `N`건에 대한 `CEE`값을 모두 구한 다음 그 값을 모두 더하고 `N`에 대한 평균을 구한다.
---
## **<center>배치를 적용한 CEE</center>**

$$
CEE = -\sum_n\sum_{k}t_{nk}\log{y}_{nk}
$$

# 미니배치란?
* `MNIST`의 데이터 개수 60,000건
* 신경망이 `MNIST`를 학습하고 거기에 대한 평가를 내릴 때 60,000건 모두에 대한 손실 함수의합을 구해야 할까?
* 데이터의 양이 굉장히 많은 경우에는 모든 데이터를 다 쓰는 것이 아니고, **데이터의 일부를 랜덤하게 추려서** 근사치로 이용할 수 있다.
* 이 일부가 되는 데이터를 **미니 배치**라고 한다.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# mnist 데이터세트 로딩
from tensorflow.keras import datasets
mnist = datasets.mnist

(X_train, t_train), (X_test, t_test) = mnist.load_data()

데이터 배열 전처리

In [ ]:
TRAIN_IMAGE_SIZE = X_train.shape[0]

X_train = X_train.reshape(TRAIN_IMAGE_SIZE, -1)
X_train.shape, t_train.shape

((60000, 784), (60000,))

`t_train` One-Hot Encoding

In [ ]:
y_train_onehot = tf.one_hot(y_train, 10).numpy()
y_train_onehot.shape

(60000, 10)

# 미니배치 구현하기

In [ ]:
import numpy as np

# 훈련 데이터 전체에서 무작위로 10장만 빼오기
train_size = X_train.shape[0] # 전체 훈련 데이터 세트 크기
batch_size = 10

# train_size에서 batch_size만큼 정수를 무작위로 추출
batch_mask = np.random.choice(train_size, batch_size) # 0~59999 중에 10개의 랜덤한 정수 인덱스를 추출
batch_mask

array([14398, 48712,  6684, 44176, 41915, 33400,  9102, 49863, 16858,
       57521])

In [ ]:
# 미니배치 추출은 원본 데이터에서 batch_mask를 씌워주면 된다.
X_batch = X_train[batch_mask]
t_batch = y_train_onehot[batch_mask]

X_batch.shape, t_batch.shape

((10, 784), (10, 10))

$$
CEE = -\frac{1}{N}\sum_n\sum_{k}t_{nk}\log{y_{nk}}
$$

In [ ]:
# ver 1 - t가 OHE이 되어 있는 형태
def cross_entropy_erorr_v1(y,t):
    delta = 1e-6

    # 배치를 사용하지 않은 경우
    if y.ndim == 1: # (10,)
        # batch_size를 1로 강제 지정
        # reshape
        t = t.reshape(1, -1)
        y = y.reshape(1, -1)
    
    batch_size = y.shape[0]

    # delta를 더한 이유 : y가 0일 때 np.log(y)는 음수 무한대가 되기 떄문
    return -np.sum( t * np.log(y+delta)) / batch_size

In [ ]:
# ver 2 - t가 OHE이 안되어 있는 형태
def cross_entropy_error_v2(y, t):
    delta = 1e-6

    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

y = np.array([[0.1,0.2,0.3,0.1,0.3],
              [0.2,0.3,0.3,0.1,0.1]])
t = np.array([2,3])

cross_entropy_error_v2(y,t)

1.753278948659991

```python
np.log(y[np.arange(batch_size), t])

ex)

batch_size = 5
t = [2, 7, 0, 9, 8]

np.arange(batch_size) = [0, 1, 2, 3, 4]

y[np.arange(batch_size), t] 코드는 다음과 같이 해석
y[[0, 1, 2, 3, 4], [2, 7, 0, 9, 8]]

y[0, 2]
y[1, 7]
y[2, 0]
y[3, 9]
y[4, 8]

y는 5개의 배치 데이터에 대한 소프트맥스의 결과물

y = [[0.0, 0.1, 0.7, 0.0, 0.0, 0.1, 0.1, 0.0, 0.0, 0.0],
     [0.0, 0.1, 0.7, 0.0, 0.0, 0.1, 0.1, 0.0, 0.0, 0.0],
     [0.0, 0.1, 0.7, 0.0, 0.0, 0.1, 0.1, 0.0, 0.0, 0.0],
     [0.0, 0.1, 0.7, 0.0, 0.0, 0.1, 0.1, 0.0, 0.0, 0.0],
     [0.0, 0.1, 0.7, 0.0, 0.0, 0.1, 0.1, 0.0, 0.0, 0.0]]

np.log(y[np.arange(batch_size), t])

* 인덱스 0번째 계산
np.log(y[0, 2]) 는 곧
np.log(0.7)
```